In [18]:
import jieba
import jieba.posseg as pseg
from pyecharts import options as opts
from pyecharts.charts import Graph


txt_file_name = './data/水浒传.txt'
node_file_name = './output/水浒传-人物节点.csv'
link_file_name = './output/水浒传-人物连接.csv'
out_file_name = './output/关系图-水浒人物.html'

with open(txt_file_name,"r+",encoding="utf-8") as f:
    line_list = f.readlines()

jieba.load_userdict('./data/userdict.txt')


In [19]:

line_name_list = []  # 每个段落出现的人物列表
name_cnt_dict = {}  # 统计人物出现次数
print('正在分段统计……')
print('已处理词数：')

ignore_list = ["言语","太尉","太公","梁山泊","寻思","相公","言语","大官人","招安","安抚",""]

progress = 0  # 用于计算进度条
for line in line_list: # 逐个段落循环处理
    word_gen = pseg.cut(line) # peseg.cut返回分词结果，“生成器”类型
    line_name_list.append([])
    
    for one in word_gen:
        word = one.word
        flag = one.flag
        
        if len(word) == 1:  # 跳过单字词
            continue
        
        if word in ignore_list:  # 跳过标记忽略的人名 
            continue
        
        # 对指代同一人物的名词进行合并
        if word == '宋江道' or word == '宋公明' or word == "公明" or word == "宋江便":
            word = '宋江'
        elif word == '高太尉':
            word = '高俅'  
        elif word == '戴宗道':
            word = '戴宗'
        elif word == '和尚':
            word = '鲁智深'
            
        if flag == 'nr': 
            line_name_list[-1].append(word)
            if word in name_cnt_dict.keys():
                name_cnt_dict[word] = name_cnt_dict[word] + 1
            else:
                name_cnt_dict[word] = 1
        
        progress = progress + 1
        progress_quo = int (progress/1000)
        progress_mod = progress % 1000 
        if progress_mod == 0: 
            #print('---已处理词数（千）：' + str(progress_quo))
            print('\r' + '-'*progress_quo + '> '\
                  + str(progress_quo) + '千', end='')
# 循环结束点        
print()
print('基础数据处理完成')

正在分段统计……
已处理词数：
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------> 241千
基础数据处理完成


In [20]:
##--- 第2步：用字典统计人名“共现”数量（relation_dict）
relation_dict = {}

# 只统计出现次数达到限制数的人名
name_cnt_limit = 100  

for line_name in line_name_list:
    for name1 in line_name:
        # 判断该人物name1是否在字典中
        if name1 in relation_dict.keys():
            pass  # 如果已经在字典中，继续后面的统计工作
        elif name_cnt_dict[name1] >= name_cnt_limit:  # 只统计出现较多的人物
            relation_dict[name1] = {}  # 添加到字典
            #print('add ' + name1)  # 测试点
        else:  # 跳过出现次数较少的人物
            continue
        
        # 统计name1与本段的所有人名（除了name1自身）的共现数量
        for name2 in line_name:
            if name2 == name1 or name_cnt_dict[name2] < name_cnt_limit:  
            # 不统计name1自身；不统计出现较少的人物
                continue
            
            if name2 in relation_dict[name1].keys():
                relation_dict[name1][name2] = relation_dict[name1][name2] + 1
            else:
                relation_dict[name1][name2] = 1

print('共现统计完成，仅统计出现次数达到' + str(name_cnt_limit) + '及以上的人物')

共现统计完成，仅统计出现次数达到100及以上的人物


In [21]:
item_list = list(name_cnt_dict.items())
item_list.sort(key=lambda x:x[1],reverse=True)
## 导出节点文件
node_file = open(node_file_name, 'w+') 
# 节点文件，格式：Name,Weight -> 人名,出现次数
node_file.write('Name,Weight\n')
node_cnt = 0  # 累计写入文件的节点数量
for name,cnt in item_list: 
    if cnt >= name_cnt_limit:  # 只输出出现较多的人物
        node_file.write(name + ',' + str(cnt) + '\n')
        node_cnt = node_cnt + 1
node_file.close()
print('人物数量：' + str(node_cnt))
print('已写入文件：' + node_file_name)

## 导出连接文件
# 共现数可以看做是连接的权重，只导出权重达到限制数的连接
link_cnt_limit = 10  
print('只导出数量达到' + str(link_cnt_limit) + '及以上的连接')

link_file = open(link_file_name, 'w+')
# 连接文件，格式：Source,Target,Weight -> 人名1,人名2,共现数量
link_file.write('Source,Target,Weight\n')
link_cnt = 0  # 累计写入文件的连接数量
for name1,link_dict in relation_dict.items():
    for name2,link in link_dict.items():
        if link >= link_cnt_limit:  # 只输出权重较大的连接
            link_file.write(name1 + ',' + name2 + ',' + str(link) + '\n')
            link_cnt = link_cnt + 1
link_file.close()
print('连接数量：' + str(link_cnt))
print('已写入文件：' + link_file_name)      

人物数量：49
已写入文件：./output/水浒传-人物节点.csv
只导出数量达到10及以上的连接
连接数量：1274
已写入文件：./output/水浒传-人物连接.csv
